In [24]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy

# Problem 1

### `1a` - a forward price & value of a forward contract 

Question: 

Let $F_t$ be the time-$t$ forward price for $T_2$-delivery of an arbitrary underlying $S$. The payout of the forward is $S_{T_2}-K$. In this case, since $F_t$ is the time-$t$ forward price, by definition $K=F_t$. So a forward contract paying $S_{T_2}-K$ at time $T_2$ has time-$t$ value 0. If $S$ is a stock paying no dividends, the forward price must be $F_t = S_t e^{r(T_2−t)}$; otherwise, arbitrage would exist.

Let $f_t$ be the time-$t$ value of a $T_2$-forward contract on the same underlying, but with some
delivery price $K$ (not necessarily equal to $F_t$).
Express $f_t$ in terms of $K$ and $F_t$ and a discount factor.

Answer:

Definitions - 
- The holder of a forward contract receives time-$T$ dellivery of underlying $S$, and pays at time $T$ a delivery price of $K$.
- $f_t$ is the time-$t$ value of a forward contract with some delivery price $K$
- $f_0$ is the time-$t$ value of a forward contract with some delivery price $F_t$
- $F_t$ is the forward price such that the forward contract paying $S_{T_2} − F_t$ at time $T_2$ has time-$t$ value 0

Since we pay delivery price $K$ at $T_2$ and get $S$, backtracking to time $t$ we attach a discount factor,
- $f_t = S_t - K e^{-r(T_2-t)}$
- $f_0 = S_t - F_t e^{-r(T_2-t)}$

Construct a porfolio $V(t) = (f_t, - f_0)$, then 
- At time-0, $V := f_t - 0 = f_t$
- At time-$t$, $V(0) = (S_t - K e^{-r(T_2-t)}) - (S_t - F_t e^{-r(T_2-t)})$
- Setting them equal, 
$$f_t = (F_t - K) e^{-r(T_2-t)}$$

### `1b` - a forward price & value of a forward contract 

Question:

If S is a stock paying no dividends, the forward price must be $F_t = S_t e^{r(T_2−t)}$; otherwise, arbitrage would exist. If, say, $F_t > S_t e^{r(T_2−t)}$, then arbitrage would exist: at time$ $t, borrow $S_t$ dollars, buy the stock, and short the forward (with delivery price $F_t$ and time-$t$ value 0). At time $T_2$, deliver the stock, and receive $F_t$, which is more than enough to cover your accumulated debt of $S_t e^{r(T_2−t)}$ dollars.

However, if $S$ is the spot price of a barrel of crude oil (so, for all $t$, the time-$t$ price for time-$t$ delivery is $S_t$ per barrel), then this argument fails. Explain briefly (one or two sentences, no math) why this specific arbitrage does not apply to crude oil, by specifically pinpointing, in the quote above, why we cannot simply replace “stock” with “crude oil”.

Answer:

Dividend is negative carry, which includes any transportation cost, storage cost, etc. Now, instead of the original formula, we have
$$F_t = S_t e^{r(T_2−t)} + \text{storage cost}$$
So even if $F_t > S_t e^{r(T_2−t)}$, arbitrage might not exist because it also needs to cover the storage cost.

### `1c` - Monte Carlo simulation of European call on forward
So we need more assumptions to relate $F_t$ and $S_t$ (here and in (`c`,`d`,`e`,`f`,`g`), the $S$ denotes spot crude oil, and $F_t$ denotes the time-$t$ forward price for $T_2$-delivery crude oil). One approach is to model the risk-neutral dynamics of $S$. Assume that $S$ satisfies
$$
\begin{aligned}S_t&=\exp(X_t)\\\mathrm{d}X_t&=\kappa(\alpha-X_t)\mathrm{d}t+\sigma\mathrm{d}W_t.\end{aligned}
$$
where $W$ is Brownian motion, under risk-neutral measure.

Then, since $r$ is constant and $\mathbb{E}_t(e^{-r(T_2-t)}(S_{T_2}-F_t))$ must be 0, one can calculate
$$
F_t=\mathbb{E}_t(S_{T_2})=\exp\left[e^{-\kappa(T_2-t)}\log S_t+(1-e^{-\kappa(T_2-t)})\alpha+\frac{\sigma^2}{4\kappa}(1-e^{-2\kappa(T_2-t)})\right],
$$

where $\mathbb{E}_t$ is time-$t$ conditional expectation. Suppose $\kappa = 0.472$, $\alpha = 4.4$, $\sigma = 0.368$, $r = 0.05$, and the time-0 spot price is $S_0 = 106.9$.


Let $C$ be the time-0 price of a $K$-strike $T_1$-expiry European call on $F$. So this call pays $(F_{T_1} − K)+$. Let the call option have strike $K = 103.2$ and expiration $T_1 = 0.5$. Let the forward have delivery date $T_2 = 0.75$. 

Question:

Estimate $C(S_0)$ using Monte Carlo simulation of $S$ with 100 timesteps on $[0,T_1]$. Choose the number of paths large enough that the standard error [the sample standard deviation, divided by the square root of the number of paths] is less than 0.05. Report the standard error. Don’t use any variance reduction technique.

In [4]:
# Exponential Ornstein-Uhlenbeck process

class XOU:

    def __init__(self, kappa, alpha, sigma, S0, r):

        self.kappa = kappa
        self.alpha = alpha
        self.sigma = sigma
        self.S0 = S0
        self.r = r

In [5]:
hw5dynamics=XOU(kappa = 0.472, alpha = 4.4, sigma = 0.368, S0 = 106.9, r = 0.05)

In [6]:
class CallOnForwardPrice:

    def __init__(self, K1, T1, T2):

        self.K1 = K1
        self.T1 = T1
        self.T2 = T2


In [7]:
hw5contract=CallOnForwardPrice(K1 = 103.2, T1 = 0.5, T2 = 0.75)

### Monte Carlo Algorithm
Let $Y$ be the discounted payoff. For example, $Y=e^{-\int_{0}^{T}r_{t}\mathrm{d}t}(S_{T}-K)^{+}$, then the Monte Carlo estimate of the time-0 price $C$ of $Y$ is:
$$\hat C_M:=\frac{Y_1+Y_2+\cdots+Y_M}M$$

In our case, 
$$
\begin{aligned}S_t&=\exp(X_t)
\\\mathrm{d}X_t&=\kappa(\alpha-X_t)\mathrm{d}t+\sigma\mathrm{d}W_t.\end{aligned}
$$
where all the constants are known, so is $S_0$.

In our algorithm, as we advance in the time steps $\mathrm{d}t = T_1/N$ from $0$ to $T_1$,
- Start with one path, the initial condition: $X_0 = \log(S_0)$
- Iterative step: $X_{t+\Delta t} = X_t + \kappa(\alpha-X_t)\mathrm{d}t+\sigma\mathrm{d}W_t$, where we approximate $\Delta W_t \approx N(0, t)$
- Final stock price: $X_{T_1}$
- Forward price: $F_{T_1}=\exp\left[e^{-\kappa(T_2-t)}X_{T_1}+(1-e^{-\kappa(T_2-t)})\alpha+\frac{\sigma^2}{4\kappa}(1-e^{-2\kappa(T_2-t)})\right]$
- Payoff of the call: $C = (F_{T_1} - K)^+$, where $K$ is the call strike price
- Repeat this $M$ times
- Monte Carlo estimate of the time-0 call price: 
$$\hat C_M:=\frac{C_1+C_2+\cdots+C_M}M$$

### `1d` - Monte Carlo simulation of call delta
Estimate $\partial C/\partial S$ by using Monte Carlo simulation to calculate $(C(S_0 + 0.01) − C(S_0))/0.01$. For the $C(S_0+0.01)$ calculation, reuse the same normal random variables which you generated for the $C(S_0)$ calculation. (Do not re-generate random variables to compute $C(S_0 + 0.01))$

In [19]:
class MCengine:

    def __init__(self, N, M, epsilon, seed):

        self.N = N   # Number of timesteps on each path
        self.M = M   # Number of paths
        self.epsilon = epsilon  # For the dC/dS calculation
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_XOU(self, contract, dynamics):
        # You complete the coding of this function
        # self.rng.normal() generates pseudo-random normals

        dt = contract.T1 / self.N
        Xs = np.ones(self.M) * np.log(dynamics.S0)
        # for C(S0+0.01) in call delta, use the same normal random variables generated for C(S0) 
        Xs2 = np.ones(self.M) * np.log(dynamics.S0 + 0.01)

        for i in range(self.N):
            # vectorization
            norms = self.rng.normal(size=self.M)  # Generate M random normal numbers
            Xs += dynamics.kappa * (dynamics.alpha - Xs) * dt + dynamics.sigma * np.sqrt(dt) * norms
            Xs2 += dynamics.kappa * (dynamics.alpha - Xs2) * dt + dynamics.sigma * np.sqrt(dt) * norms
        # print(Xs) # final Xs at T1

        # Calculate the forward prices at T1
        Fs = np.exp(np.exp(-dynamics.kappa * (contract.T2 - contract.T1)) * Xs 
                    + (1 - np.exp(-dynamics.kappa * (contract.T2 - contract.T1))) * dynamics.alpha
                    + dynamics.sigma**2 / (4 * dynamics.kappa) * (1 - np.exp(-2 * dynamics.kappa * (contract.T2 - contract.T1))))
        Fs2 = np.exp(np.exp(-dynamics.kappa * (contract.T2 - contract.T1)) * Xs2 
                    + (1 - np.exp(-dynamics.kappa * (contract.T2 - contract.T1))) * dynamics.alpha
                    + dynamics.sigma**2 / (4 * dynamics.kappa) * (1 - np.exp(-2 * dynamics.kappa * (contract.T2 - contract.T1))))
        # print(Fs)
        print("The price of foward F at T1 is ", np.mean(Fs))

        call_payoffs = np.maximum(Fs - contract.K1, 0)
        call_payoffs2 = np.maximum(Fs2 - contract.K1, 0)
        # print(call_payoffs)

        # Calculate the price of the call option, based on the average of discounted payoffs
        call_price = np.mean(call_payoffs) * np.exp(-dynamics.r * contract.T1)
        call_price2 = np.mean(call_payoffs2) * np.exp(-dynamics.r * contract.T1)

        standard_error = np.std(call_payoffs) / np.sqrt(self.M)
        call_delta = (call_price2 - call_price) / 0.01

        return (call_price, standard_error, call_delta)


In [20]:
hw5MC = MCengine(N=100, M=100000, epsilon=0.01, seed=0)
# Change M if necessary

In [21]:
(call_price, standard_error, call_delta) = hw5MC.price_call_XOU(hw5contract,hw5dynamics)

The price of foward F at T1 is  102.16096789172634


In [22]:
print(call_price, standard_error, call_delta)

7.741671334651303 0.0430646450926069 0.34111144701913787


### `1e` - forward-value delta
Question: 

Calculate analytically $\partial f_0/\partial S$, where $f_0$ is the time-0 value of a position long one forward contract on a barrel of crude oil, with delivery date $T_2$ and some fixed delivery price $K$.

Using answer from part `a`,
$$\begin{aligned}
\frac{\partial f_0}{\partial S} &= \frac{\partial \left[(F_0 - K)e^{-r T_2}\right]}{\partial S}\\
&= e^{-r T_2} \frac{\partial F_0}{\partial S} \\
&= e^{-r T_2} \frac{\partial \exp\left[e^{-\kappa(T_2)}\log S+(1-e^{-\kappa(T_2)})\alpha+\frac{\sigma^2}{4\kappa}(1-e^{-2\kappa(T_2)})\right]}{\partial S} \\
&= e^{-r T_2} \exp\left[e^{-\kappa(T_2)}\log S+(1-e^{-\kappa(T_2)})\alpha+\frac{\sigma^2}{4\kappa}(1-e^{-2\kappa(T_2)})\right] \frac{\partial e^{-\kappa(T_2)}\log S}{\partial S} \\
&= \exp\left[e^{-\kappa T_2}\log S+(1-e^{-\kappa T_2})\alpha+\frac{\sigma^2}{4\kappa}(1-e^{-2\kappa T_2})\right] \frac{e^{-(r+\kappa) T_2}}{S} \\
\end{aligned}
$$

### `1f` - hedge position
Question:

Suppose you want to hedge a position short one call (so your hedge portfolio should replicate a position long one call), by continuously rebalancing a position in $T_2$-delivery forward contracts. Your hedge portfolio at time 0 should be long how many forward contracts? Your final answer should be a number.

The delivery price $K$ of the forward contracts is irrelevant to the answer here; it would affect only how many units of the bank account to carry in the portfolio (which I am not asking you to compute).

Answer:

To hedge a position short one call option by continuously rebalancing a position in $T_2$-delivery forward contracts, we need to determine the initial delta of the call option with respect to the underlying asset and then use that to determine the number of forward contracts required.

For a forward contract, the delta with respect to the spot price is 1 because the value of a forward contract moves one-to-one with the spot price. Therefore, the hedge ratio (number of forward contracts required to hedge one call option) is equal to the delta of the call option, which is 0.341 in our case.

### `1g` - purchase agreement contract
Question: 

Consider the following “purchase agreement” contract. The holder of this contract receives time-$T_2$ delivery of $\theta$ barrels of crude oil, and pays, at time $T_2$, a delivery price of $K$ dollars per barrel. The $\theta$ is chosen at time $T_1$ by the holder of the purchase agreement, subject to the restriction that $4000 \leq \theta \leq 5000$; in particular, $\theta = 0$ is not a valid choice, because the contract is a commitment to purchase at least 4000 barrels. Using your answer to (c), without running any new simulations, find the time-0 value of this contract.

Here $K$, $T_1$, $T_2$ have the same values as on the previous page. Hint: Assume the holder acts optimally; thus $\theta$ is either 4000 or 5000, depending on $F_{T_1}$ .

Answer:

Since the price of forward at $T_1$, according to our Monte Carlo simulation, is $F_{T_1} = 102.16 < K = 103.2$, we pick $\theta=5000$, because the forward price is low in comparison.

# Problem 2

Suppose that a non-dividend-paying stock has dynamics
$$dS_t = rS_tdt + \sigma(t)S_tdW_t$$
where $W$ is Brownian motion under risk-neutral probabilities, and where the time-dependent but non-random volatility function $\sigma : [0,T] \rightarrow R$ is piecewise continuous and sufficiently integrable. L2.13 shows that this particular type of local volatility function σ (to be specific: the type of σ function that depends on t but does not depend on S, nor on anything else that is random) has an explicit relationship with the Black-Scholes implied volatility σimp.


### `2a` - implied volatility term-structure, implied volatility skew
1. Question: Are the dynamics (1) capable of generating a non-constant (with respect to $T$) term-structure of implied volatility? Answer: Yes. Since the instantaneous volatility $\sigma(t)$ is time-dependent, the volatility over different expiries $T$ can vary, creating a non-constant term structure.
1. Question: Are they capable of generating an implied volatility skew (non-constant with respect to $K$)? Answer: No. An instantaneous volatility $\sigma(t)$ cannot generate an impiled volatility skew because it does not depend on stock price $S$.

### `2b` - implied volatility
Question: 

Let $S_0 = 100$ and $r = 0.05$. At time 0, you observe the prices of at-the-money (this means $K = 100$) European calls at 0.1-year, 0.2-year, and 0.5-year expiries to be 5.25, 7.25, and 9.5, respectively. First find the Black-Scholes implied volatilities of the three options. 

Answer:
- Black-Scholes Vanilla Call pricing:
$$C(S,t)=C^{BS}(S,t,K,T,r-q,r,\sigma)=S_0N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}+\frac{\sigma\sqrt T}2\right)-Ke^{-rT}N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}-\frac{\sigma\sqrt T}2\right)$$
- In our case, have
$$5.25=100 N\left(\frac{\log(100 e^{0.05 \times 0.1}/100)}{\sigma_1\sqrt {0.1}}+\frac{\sigma_1\sqrt {0.1}}2\right)-100e^{-0.05\times 0.1}N\left(\frac{\log(100e^{0.05\times 0.1}/100)}{\sigma_1\sqrt {0.1}}-\frac{\sigma_1\sqrt {0.1}}2\right)$$
$$7.25=100 N\left(\frac{\log(100 e^{0.05 \times 0.2}/100)}{\sigma_2\sqrt {0.2}}+\frac{\sigma_2\sqrt {0.2}}2\right)-100e^{-0.05\times 0.2}N\left(\frac{\log(100e^{0.05\times 0.2}/100)}{\sigma_2\sqrt {0.2}}-\frac{\sigma_2\sqrt {0.2}}2\right)$$
$$9.5=100 N\left(\frac{\log(100 e^{0.05 \times 0.5}/100)}{\sigma_3\sqrt {0.5}}+\frac{\sigma_3\sqrt {0.5}}2\right)-100e^{-0.05\times 0.5}N\left(\frac{\log(100e^{0.05\times 0.5}/100)}{\sigma_3\sqrt {0.5}}-\frac{\sigma_3\sqrt {0.5}}2\right)$$

In [26]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [23]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

In [27]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def IV(self, dynamics, contract):
        # ignores dynamics.sigma, because this function solves for sigma.

        if contract.price is None:
            raise ValueError('Contract price must be given')

        df = np.exp(-dynamics.r*contract.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above.
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval,
        # for purposes of this homework, you may either (A) write your own bisection algorithm,
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).
        # However, if you are able to bound the solution (of a single-variable problem),
        # then bisection or Brent will be more reliable.

        # Option B: use scipy.optimize.bisect
        # Need to pass the function that takes the implied volatility as a single argument
        # Define a function for the call price as a function of sigma
        def func(sigma):
            return self.BSpriceCall(dytry.update_sigma(sigma), contract) - C

        impliedVolatility = bisect(func, lo, hi)

        return impliedVolatility


In [33]:
#Test the BSpriceCall function
hw5analytic = AnalyticEngine()

dynamics1 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0.05)
contract1 = CallOption(K=100, T=0.1)
contract1.price = 5.25

dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0.05)
contract2 = CallOption(K=100, T=0.2)
contract2.price = 7.25

dynamics3 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0.05)
contract3 = CallOption(K=100, T=0.5)
contract3.price = 9.5

#Test the IV function
hw5IV1 = hw5analytic.IV(dynamics1,contract1)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
hw5IV2 = hw5analytic.IV(dynamics2,contract2)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
hw5IV3 = hw5analytic.IV(dynamics3,contract3)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
print(f"IV for a European call on S at 0.1-year expiry: {hw5IV1}")
print(f"IV for a European call on S at 0.2-year expiry: {hw5IV2}")
print(f"IV for a European call on S at 0.5-year expiry: {hw5IV3}")

IV for a European call on S at 0.1-year expiry: 0.4185406162767322
IV for a European call on S at 0.2-year expiry: 0.4110237902568769
IV for a European call on S at 0.5-year expiry: 0.34615367469523345


Question: 

Then find (calibrate) a time-varying local volatility function $\sigma : [0, 0.5] \rightarrow R$ consistent with these option prices. A step function suffices (but other answers are also acceptable).

Answer:
- We approximate the step function:
\begin{aligned}
\sigma(t) \approx
0.419 & \text{ for } t \in [0, 0.1] \\
0.411 & \text{ for } t \in (0.1, 0.2] \\
0.346 & \text{ for } t \in (0.2, 0.5] \\
\end{aligned} 

### `2c` - European call price implied volatility
Question: 

Consistently with your local volatility function σ from part (b), find the time-0 price of an at-the-money European call with expiry 0.4, and find the time-0.1 implied volatility of that call (the European call with expiry $T = 0.4$). Do not use a tree or finite difference or Monte Carlo calculation.

Answer:
- The expiry $T = 0.4$ calls in the last bin of our step function, so we plug in $\sigma=0.346$ in our pricing model.
- Black-Scholes Vanilla Call pricing:
$$C(S,t)=C^{BS}(S,t,K,T,r-q,r,\sigma)=S_0N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}+\frac{\sigma\sqrt T}2\right)-Ke^{-rT}N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}-\frac{\sigma\sqrt T}2\right)$$

In [36]:
dynamics4 = GBMdynamics(sigma=0.346, rGrow=0, S=100, r=0.05)
contract4 = CallOption(K=100, T=0.4)

call_price4 = hw5analytic.BSpriceCall(dynamics4, contract4)
print(f"The time-0 price of an at-the-money European call with expiry 0.4 is: {call_price4}")

The time-0 price of an at-the-money European call with expiry 0.4 is: 8.540128957406202


In [37]:
contract5 = CallOption(K=100, T=0.1)
contract5.price = call_price4

#Test the IV function
hw5IV5 = hw5analytic.IV(dynamics4, contract5)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
print(f"IV for this European call on S at 0.1-year expiry: {hw5IV5}")


IV for this European call on S at 0.1-year expiry: 0.6816571717485205
